# Introduzione

Mario Fiorino

Come costruire un agente capace di giocare a Tris (Tic-Tac-Toe Game) usando un approccio basato sul Reinforcement learning?

In questo notebook è stato implementato un agente Q-learning che impara a giocare a tris

*Cosa si  intende per Q-learning?*

Q-learning è un algoritmo  model-free, value-based, off-policy ( cioè che valuta e migliora una policy non necessariamente uguale quella utilizzata per il controllo ) temporal-difference[*] control, che consente di modellare un agente a prendere decisioni ottimali in un ambiente non necessariamente noto a priori. L'agente apprende la funzione di valore Q (che può essere una struttura dati semplice come una tabella, o qualcosa di più articolato come una rete neurale) che rappresenta il valore atteso di un'azione in un determinato stato.


[*]NOTA:  a differenza dei metodi Monte Carlo che aspettano fino alla fine di un episodio per aggiornare le loro stime. I metodi di distanza temporale, invece, aggiornano le loro stime man mano che l'agente procede nell'episodio

Ispirazione codice :

https://ai.plainenglish.io/building-a-tic-tac-toe-game-with-reinforcement-learning-in-python-a-step-by-step-tutorial-5a6d9bcbb764

E' stato usato come ambiente di sviluppo integrato (IDE) basato su cloud: COLAB

Alcuni strumenti per la generazione di numeri random "veri" :

https://pynative.com/python-secrets-module/

https://www.geeksforgeeks.org/secrets-python-module-generate-secure-random-numbers/




# Moduli

In [1]:
#import tensorflow as tf
import numpy as np

import random
import secrets # This module is responsible for providing access to the most secure source of randomness

import pandas as pd

import pickle

# Struttura del gioco Tris

Il gioco del tris si gioca su una griglia 3x3, con due giocatori che si alternano nel contrassegnare una casella con il loro simbolo (X o O). L'obiettivo del gioco è ottenere tre dei propri simboli in fila, orizzontalmente, verticalmente o diagonalmente

In [2]:
class Tris:

    # Inizializza variabili del gioco
    def __init__(self):

        self.board = np.zeros((3, 3))
        #  inizializza il contenuto della griglia (questo non sarà output grafico)
        # [[0. 0. 0.]
        #  [0. 0. 0.]
        #  [0. 0. 0.]]

        self.players = ['X', 'O']   # Due giocatori contrassegnano una casella con X oppure O. Questo punto riguarda la parte grafica
        self.current_player = None
        self.winner = None
        self.game_over = False


    # Questo metodo reimposta la tastiera di gioco, il giocatore corrente, il vincitore e
    # lo stato di fine gioco ai loro valori iniziali.
    def reset(self):
        self.board = np.zeros((3, 3))
        self.winner = None
        self.game_over = False


    def available_moves(self):
        moves = []
        for i in range(3):
            for j in range(3):
                if self.board[i][j] == 0:
                    moves.append((i, j))
        # oppure in comprehension list :
        # moves = [(i, j) for i in range(3) for j in range(3) if self.board[i][j] == 0]

        return moves   # ritorna la lista dei possibli movimenti, cioè dove le caselle sono libere ed hanno valore 0


    def make_move(self, move):
                      # move è una tupla con le cooridnate della casella in cui vuole effettuata la mossa.

        if self.board[move[0]][move[1]] != 0:
            return False  # Se la casella è già occupata, restituisce False, indicando che la mossa non è valida

        #Aggiorna il tabellone con il simbolo del giocatore attuale,
        #controlla se la mossa ha portato a una vittoria
        #passa al turno dell'altro giocatore.
        self.board[move[0]][move[1]] = self.players.index(self.current_player) + 1
        # Note:  self.players.index("X") = 0 mentre self.players.index("O") = 1

        self.check_winner()
        self.switch_player()
        return True


    def switch_player(self):
        if self.current_player == self.players[0]:
            self.current_player = self.players[1]
        else:
            self.current_player = self.players[0]


    def check_winner(self):
        # Controllo riga
        for i in range(3):
            if self.board[i][0] == self.board[i][1] == self.board[i][2] != 0:
                #Se c'è un vincitore, imposta di conseguenza
                #il vincitore e
                #lo stato di game over.
                self.winner = self.players[int(self.board[i][0] - 1)]
                self.game_over = True
        # Controllo colonna
        for j in range(3):
            if self.board[0][j] == self.board[1][j] == self.board[2][j] != 0:
                self.winner = self.players[int(self.board[0][j] - 1)]
                self.game_over = True
        # Controllo diagonali
        if self.board[0][0] == self.board[1][1] == self.board[2][2] != 0:
            self.winner = self.players[int(self.board[0][0] - 1)]
            self.game_over = True
        if self.board[0][2] == self.board[1][1] == self.board[2][0] != 0:
            self.winner = self.players[int(self.board[0][2] - 1)]
            self.game_over = True


    #Output grafico della griglia
    #Questo metodo stampa lo stato corrente della griglia
    def print_board(self):
        print("-------------")
        for i in range(3):
            print("|", end=' ')
            for j in range(3):
                print(self.players[int(self.board[i][j] - 1)] if self.board[i][j] != 0 else " ", end=' | ')
            print()
            print("-------------")

# Test della classe Tris

Proviamo la classe appena creata, generando un'istanza della classe e giocando manualmente.  

X è un utente che inserisce con input() la propria mossa.

O è una funzione random.choice()

In ogni iterazione del loop, richiede al giocatore corrente di inserire la propria mossa, controlla se la mossa è valida ed esegue la mossa. Dopo ogni mossa, stampa lo stato aggiornato della griglia.

In [26]:
game = Tris()
game.current_player = game.players[0] # Imposta il giocatore corrente su X. Quindi sarà lui a far la prima mossa. Nota : game.players[0] == "X"
game.print_board()

while (not game.game_over) and (bool(game.available_moves())) :
    # fin quando non ci sono vincitori : (not game.game_over) == True
    # e
    # fin quando non ci sono azioni possibli : (bool(game.available_moves())) == True
    # Note : bool([]) == False
    # stai nel loop ...
    print("Azioni possibili, espresse in coordinate: " , game.available_moves()  )

    if game.current_player == game.players[0] :
       move = input(f"{game.current_player} è il tuo turno. Inserisci riga e colonna (e.g. 0 0): ")
       move = tuple(map(int, move.split()))
       # come lavora map()
       while move not in game.available_moves():
         move = input("Mossa non valida, riprova: ")
         move = tuple(map(int, move.split()))
    else:
        move = random.choice(game.available_moves())

    print("Mossa scelta : " , move)

    game.make_move(move)
    game.print_board()
    print(" ")

if game.winner:
    print(f"{game.winner} Vince!")
else:
    print("Pareggio!")

-------------
|   |   |   | 
-------------
|   |   |   | 
-------------
|   |   |   | 
-------------
Azioni possibili, espresse in coordinate:  [(0, 0), (0, 1), (0, 2), (1, 0), (1, 1), (1, 2), (2, 0), (2, 1), (2, 2)]
X è il tuo turno. Inserisci riga e colonna (e.g. 0 0): 0 0
Mossa scelta :  (0, 0)
-------------
| X |   |   | 
-------------
|   |   |   | 
-------------
|   |   |   | 
-------------
 
Azioni possibili, espresse in coordinate:  [(0, 1), (0, 2), (1, 0), (1, 1), (1, 2), (2, 0), (2, 1), (2, 2)]
Mossa scelta :  (2, 2)
-------------
| X |   |   | 
-------------
|   |   |   | 
-------------
|   |   | O | 
-------------
 
Azioni possibili, espresse in coordinate:  [(0, 1), (0, 2), (1, 0), (1, 1), (1, 2), (2, 0), (2, 1)]
X è il tuo turno. Inserisci riga e colonna (e.g. 0 0): 1 1
Mossa scelta :  (1, 1)
-------------
| X |   |   | 
-------------
|   | X |   | 
-------------
|   |   | O | 
-------------
 
Azioni possibili, espresse in coordinate:  [(0, 1), (0, 2), (1, 0), (1, 2), (2,

# Implementazione  "Reinforcement Learning Agent"

Qui sarà implementato un agente, che tramite una tecnica di RL, l'algoritmo Q-learning, imparerà a vincere (troverà la politica ottimale per ciascuna coppia stato-azione) a Tris.



In [36]:
class QLearningAgent:
    def __init__(self, alpha, epsilon, discount_factor):

        self.Q = {} # la struttura dati per rappresentare la Q-table è un dizionario :
                    #  chiave: (stato,azione) -> valore:  Q-values.

        #paramentri
        self.alpha = alpha      #  learning rate, controlla quanto i valori Q vengono aggiornati ad ogni passaggio.
        self.epsilon = epsilon  #  exploration rate, che controlla la probabilità di scegliere un'azione casuale invece dell'azione ottimale.
        self.discount_factor = discount_factor


    def get_Q_value(self, state, action):
      # Questo metodo restituisce il valore Q per una determinata coppia stato-azione.
      # Chiavi del dizianario Q : stato, azione
      # Ogni stato è una tupla che rappresenta lo stato corrente della griglia
      # Esempio lo stato iniziale : [0. 0. 0. 0. 0. 0. 0. 0. 0.]
      # Ogni azione è una tupla che rappresenta le coordinate del movimento.
      # Esempio posizione al centro della griglia : (1, 1)
      # Valore del dizionario Q : Q-value, ovvero i valori rincompensa
        #print("INPUT state in 'get_Q_value'",state)
        #print("INPUT action in 'get_Q_value'",action)
        if (state, action) not in self.Q:
            self.Q[(state, action)] = 0.0   # I valori Q iniziali verranno impostati su zero
        #print("OUTPUT of method 'get_Q_value',the Current dict Q=",self.Q )
        return self.Q[(state, action)]

    def choose_action(self, state, available_moves):
        if  secrets.SystemRandom().uniform(0,1) < self.epsilon: # oppure: random.uniform(0, 1) < self.epsilon:
            return secrets.choice(available_moves)
        else:
            # sceglie l'azione con il valore Q più alto.
            Q_values = []
            for action in available_moves:
                 Q_values.append(self.get_Q_value(state, action))
            #print( "La lista dei valori Q_values per le azioni ancora disponibili (in method 'choose_action') :\n ",Q_values)
            max_Q = max(Q_values)

            if Q_values.count(max_Q) > 1:
                best_moves = [i for i in range(len(available_moves)) if Q_values[i] == max_Q]
                i = random.choice(best_moves)
            else:
                i = Q_values.index(max_Q)
            return available_moves[i]

    def update_Q_value(self, state, action, reward, next_state):
        # Questo metodo aggiorna il valore Q per una determinata coppia stato-azione in base all'algoritmo Q-learning.

        # Sotto la lista di tutti i valori Q che potrebbero essere ottenuti partendo dallo stato 'next_action', applicando le azioni disponibili in questo stato.
        next_Q_values = [self.get_Q_value(next_state, next_action) for next_action in Tris().available_moves()]

        if not next_Q_values : # se la lista è vuota  ritorna  0
            max_next_Q = 0.0
        else:
            max_next_Q =  max(next_Q_values)    # altrimenti ritorno il massimo dei valori futuri ancora possibili

        # Processo iterativo di aggiornamento e correzione basato sulla nuova informazione.
        self.Q[(state, action)] = self.Q[(state, action)] + self.alpha * (reward + self.discount_factor * max_next_Q - self.Q[(state, action)])
        #print(f"Sone nel metodo 'update_Q_value'\n Valore aggiornato di Q in stato {state} per l'azione {action} è : {self.Q[(state, action)]} ")

        return self.Q

# Test della classe QlearningAgent

In [ ]:
alpha=0.5
epsilon=0.1
discount_factor=1.0

agent = QLearningAgent(alpha, epsilon, discount_factor)

state = Tris().board

available_moves = Tris().available_moves()
print("available_moves=", available_moves)
print("state=\n",state)

# Per lavorare con una struttura dati dizionario
# c'è bisogno che stato attuale della griglia sia hashable e non un 'numpy.ndarray'
boardHash = str(state.reshape(3 * 3))
print("boardHash=",boardHash)

action = agent.choose_action(boardHash, available_moves)
print("action=",action)

state[action[0],action[1]] = 1
print(" state after action (or next_state) =\n",state)
next_state = state

w = True
if w : # se c'è un vincitore, vedi def check_winner
  reward = 1
else :
  reward = 0

next_boardHash = str(next_state.reshape(3 * 3))
dq = agent.update_Q_value(boardHash, action, reward, next_boardHash)

sorted_dq = sorted(dq.items(), key=lambda x:x[1], reverse=True)
dq = dict(sorted_dq)
print(dq)

available_moves= [(0, 0), (0, 1), (0, 2), (1, 0), (1, 1), (1, 2), (2, 0), (2, 1), (2, 2)]
state=
 [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
boardHash= [0. 0. 0. 0. 0. 0. 0. 0. 0.]
La lista dei valori Q_values per le azioni ancora disponibili (in method 'choose_action') :
  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
action= (1, 2)
 state after action (or next_state) =
 [[0. 0. 0.]
 [0. 0. 1.]
 [0. 0. 0.]]
Metodo 'update_Q_value'
 Valore aggiornato di Q in stato [0. 0. 0. 0. 0. 0. 0. 0. 0.] per l'azione (1, 2) è : 0.5 
{('[0. 0. 0. 0. 0. 0. 0. 0. 0.]', (1, 2)): 0.5, ('[0. 0. 0. 0. 0. 0. 0. 0. 0.]', (0, 0)): 0.0, ('[0. 0. 0. 0. 0. 0. 0. 0. 0.]', (0, 1)): 0.0, ('[0. 0. 0. 0. 0. 0. 0. 0. 0.]', (0, 2)): 0.0, ('[0. 0. 0. 0. 0. 0. 0. 0. 0.]', (1, 0)): 0.0, ('[0. 0. 0. 0. 0. 0. 0. 0. 0.]', (1, 1)): 0.0, ('[0. 0. 0. 0. 0. 0. 0. 0. 0.]', (2, 0)): 0.0, ('[0. 0. 0. 0. 0. 0. 0. 0. 0.]', (2, 1)): 0.0, ('[0. 0. 0. 0. 0. 0. 0. 0. 0.]', (2, 2)): 0.0, ('[0. 0. 0. 0. 0. 1. 0. 0. 0.]', (0, 0)): 0.0, ('[0. 0.

# Fase di Training

In [ ]:
game = Tris()
# Ricorda gli stati possibili nel gioco sono 3^9 = 19683
# 9 caselle
# vuoto, marcato X, marcato O.

num_episodes = 80_000

alpha=0.5
epsilon=0.05
discount_factor=1.0

game.current_player = game.players[0]
#              Nota : game.players[0] == "X"  è il nostro agente e farà la prima mossa


agent = QLearningAgent(alpha, epsilon, discount_factor)


for i in range(1,num_episodes + 1):
     print("\nEpisode nr : ", i)

     game.reset() # pulisce la griglia dalle operazioni precedenti per poter iniziare un nuovo episodio
     state = game.board
     stateHash = str(state.reshape(3 * 3)) # Ricorda : c'è bisogno che stato attuale della griglia sia hashable e non un 'numpy.ndarray'

     #game.print_board()

     # The exploration-exploitation trade-off
     # Qui un sempllice tunning per bilanciare esplorazione e sfruttamento
     if i > 40000:
        epsilon=0.5
     elif i > 70000:
        epsilon=0.8

     while (not game.game_over) and (bool(game.available_moves())) :

         #STEP 1
         #Seleziona un’azione possibile in un certo stato ed eseguila
         print("\n***Muova mossa****\n")
         print("Azioni possibili : " , game.available_moves()  )

         print("Sta muovendo il giocatore : ",game.current_player)
         if game.current_player == game.players[0] :
              move = agent.choose_action(stateHash, game.available_moves())
              #print("Azione scelta dal Q_learning_Agent=",move)
         else:
              move = secrets.choice(game.available_moves())

         game.make_move(move)

         # Stampa a schermo la situazione di gioco della griglia dopo l'azione 'move'
         # game.print_board()

         # STEP 2
         # agente riceve la ricompensa
         # game.winner puo essere "X" o "O"
         if game.winner == 'X':
            print(f"\t\t{game.winner} Vince!")
            reward = 1.0
         elif game.winner == 'O':
            print(f"\t\t{game.winner} Vince!")
            reward = - 0.05
         else:
            reward = 0.0

         # STEP 3
         #Qui si costruisce ed aggiorna la struttura dati  Q
         next_state = game.board   # 'next_state' è il nuovo stato in cui si arriva eseguendo azione: 'action'
         next_boardHash = str(next_state.reshape(3 * 3))


         dq = agent.update_Q_value(stateHash, move, reward, next_boardHash)



print("\nSTRUTTURA DATI Q_TABLE (in pratica un dict) ottenuta alla fine del training")
#print(dq)
print("Dimensoni : ",len(dq))

#new = pd.DataFrame.from_dict(dq,orient ='index')
#print("\n")
#new

#
# Salva la struttura dati ottenuta su un file :
#
with open('Tris_data_train.pkl', 'wb') as fp:
    pickle.dump(dq, fp)
    print('dictionary saved successfully to file')

# Riprendi i dati del file salvati

In [ ]:
from google.colab import files
uploaded = files.upload()

with open('Tris_data_train.pkl', 'rb') as fp:
    dq_in_file = pickle.load(fp)

print(len(dq_in_file))
#dp = dq_in_file

# Test 1 delle prestazioni dell'agente Q-learning

Dopo aver addestrato l'agente Q-learning, possiamo testarne le prestazioni prima contro un giocatore umano.

In [ ]:
game = Tris()

game.current_player = game.players[0] # Nota : game.players[0] == "X", imposta il giocatore corrente su X e sarà lui a fare la prima mossa
# In questo caso, game.players[0] è il Q_learning_Agent
#Alternativa,
#ogni giocatore ha il 50% di possibilità di essere il primo:
#game.current_player = random.choice(game.players)

#game.print_board()

while (not game.game_over) and (bool(game.available_moves())) :

    state = game.board
    stateHash = str(state.reshape(3 * 3))

    print("Azioni possibili, espresse in coordinate: " , game.available_moves()  )

    if game.current_player == game.players[1] :
       move = input(f"{game.current_player} è il tuo turno. Inserisci riga e colonna (e.g. 0 0): ")
       move = tuple(map(int, move.split()))
       while move not in game.available_moves():
         move = input("Mossa non valida, riprova: ")
         move = tuple(map(int, move.split()))

       # Se si volesse lavorare con un agente random non umano usare:
       # move = secrets.choice(game.available_moves())

    else:
        # Agente_Q ha in input lo state e sceglie l'azione con Q-value più altro

        # Sotto costruisco un sotto-dizionario contente le sole azioni disponibili per un certo stato dato in input
        sub_dq = {}
        for i in game.available_moves():
          #print(i)
          sub_dq.update({(stateHash,i) : dq[stateHash,i]})

        newdq = pd.DataFrame.from_dict(sub_dq,orient ='index')
        print("Sub-dict delle azioni disponibili in un certo stato : \n ",newdq)

        # ne ricavo la key del dizionario che ha value Q massimo
        index_max_Qvalue = max(sub_dq, key=sub_dq.get)
        print("Indice con value Q massimo : ",index_max_Qvalue )

        # Recupera l'azione da fare
        move =  index_max_Qvalue[1]
        #print(move)


    game.make_move(move)
    game.print_board()
    print(" \nFINE MOSSA\n ")


if game.winner:
    print(f"{game.winner} Vince!")
else:
    print("Pareggio!")

# Test 2 delle prestazioni dell'agente Q-learning

Possiamo testarne le prestazioni contro un giocatore-agente random.

In [59]:
num_games = 10_000

num_wins = 0

game = Tris()

game.current_player = random.choice(game.players)


for j in range(num_games):

        game.reset()
        state = game.board
        stateHash = str(state.reshape(3 * 3))

        while (not game.game_over) and (bool(game.available_moves())) :

             if game.current_player == game.players[1] :
              # Agente_Random
                  move = secrets.choice(game.available_moves())

             else:
              # Agente_Q
                sub_dq = {}
                for i in game.available_moves():
                  sub_dq.update({(stateHash,i) : dq[stateHash,i]})

                newdq = pd.DataFrame.from_dict(sub_dq,orient ='index')
                #print("Sub-dict delle azioni disponibili in un certo stato : \n ",newdq)


                index_max_Qvalue = max(sub_dq, key=sub_dq.get)
                move =  index_max_Qvalue[1]

             game.make_move(move)
             #game.print_board()

        #print("Fine partita nr : ", j )
        #print("WINNER : ",  game.winner)

        if game.winner == game.players[0] :
           num_wins += 1

print("\nESITO FINALE DEL TEST :")
print(num_wins / num_games * 100)


ESITO FINALE DEL TEST :
69.04


# Conclusione

Usando la struttura dati ottenuta da questo veloce addestramento: 'Tris_data_train.pkl'; il nostro agente Q_learning vince quasi il 70% delle volte contro un agente_random